In [20]:
import requests
import json
from secrets import ASTROBIN_KEY, ASTROBIN_SECRET

In [115]:
astrobin_url = 'http://www.astrobin.com/api/v1/image'
base_params = {'api_key': ASTROBIN_KEY, 'api_secret': ASTROBIN_SECRET}

In [12]:
#dict(base_params, **{'subjects':'Jupiter'})

In [104]:
jupiter_search_params = {'title__icontains': 'jupiter'}
jupiter_search_params.update(base_params)

In [105]:
jupiter_search_request = requests.get(astrobin_url, params=jupiter_search_params)

In [106]:
results = json.loads(jupiter_search_request.text)

In [107]:
results['objects']

[{u'animated': False,
  u'description': u'',
  u'h': 350,
  u'id': u'230474',
  u'imaging_cameras': [],
  u'imaging_telescopes': [],
  u'is_final': True,
  u'is_solved': False,
  u'license': 0,
  u'link': u'',
  u'link_to_fits': u'',
  u'locations': [],
  u'resource_uri': u'/api/v1/image/230474/',
  u'revisions': [],
  u'subjects': [],
  u'title': u'Jupiter returning',
  u'updated': u'2015-12-06T23:15:14.856867',
  u'uploaded': u'2015-12-06T23:14:34.446210',
  u'url_duckduckgo': u'http://www.astrobin.com/230474/0/rawthumb/duckduckgo/',
  u'url_duckduckgo_small': u'http://www.astrobin.com/230474/0/rawthumb/duckduckgo_small/',
  u'url_gallery': u'http://www.astrobin.com/230474/0/rawthumb/gallery/',
  u'url_hd': u'http://www.astrobin.com/230474/0/rawthumb/hd/',
  u'url_real': u'http://www.astrobin.com/230474/0/rawthumb/real/',
  u'url_regular': u'http://www.astrobin.com/230474/0/rawthumb/regular/',
  u'url_thumb': u'http://www.astrobin.com/230474/0/rawthumb/thumb/',
  u'user': u'rg55',
  

In [108]:
len(_)

20

In [109]:
results['meta']

{u'limit': 20,
 u'next': u'/api/v1/image/?api_secret=739405ad5d65a99d309bc12b8e010b607dece4c2&title__icontains=jupiter&api_key=ef2b1a21d6846a06cfce6ef9e73b85d1c9e84aa0&limit=20&offset=20',
 u'offset': 0,
 u'previous': None,
 u'total_count': 7489}

In [119]:
image_search_request = requests.get(astrobin_url+'/230132/', params=base_params)

In [121]:
json.loads(image_search_request.text)

{u'animated': False,
 u'description': u'100%.',
 u'h': 294,
 u'id': u'230132',
 u'imaging_cameras': [u'iDS CMOS UI-1240LE-C-HQ'],
 u'imaging_telescopes': [u'Celestron SCT 9.25'],
 u'is_final': True,
 u'is_solved': False,
 u'license': 0,
 u'link': u'',
 u'link_to_fits': u'',
 u'locations': [],
 u'resource_uri': u'/api/v1/image/230132/',
 u'revisions': [],
 u'subjects': [],
 u'title': u'Jupiter',
 u'updated': u'2015-12-05T15:38:24.927918',
 u'uploaded': u'2015-12-05T15:37:27.486101',
 u'url_duckduckgo': u'http://www.astrobin.com/230132/0/rawthumb/duckduckgo/',
 u'url_duckduckgo_small': u'http://www.astrobin.com/230132/0/rawthumb/duckduckgo_small/',
 u'url_gallery': u'http://www.astrobin.com/230132/0/rawthumb/gallery/',
 u'url_hd': u'http://www.astrobin.com/230132/0/rawthumb/hd/',
 u'url_real': u'http://www.astrobin.com/230132/0/rawthumb/real/',
 u'url_regular': u'http://www.astrobin.com/230132/0/rawthumb/regular/',
 u'url_thumb': u'http://www.astrobin.com/230132/0/rawthumb/thumb/',
 u'us